In [2]:
#import modules

#work with data as table
import pandas as pd

#use number matrices
import numpy as np

#visualizations
import matplotlib.pyplot as plt

#tensorflow
import tensorflow as tf

In [3]:
#step 1: Load Data
dataframe = pd.read_csv('/home/nbuser/data.csv') #dataframe

#remove unrequired features
dataframe = dataframe.drop(['index','price','sq_price'], axis=1)

#use only first ten rows
dataframe = dataframe[0:10]

In [4]:
#view our data
dataframe

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


In [9]:
#step 2 - Add labels
#1 is good buy and 0 is bad
dataframe.loc[:, ('y1')] = [1,1,1,0,0,1,1,1,1,1]

#y2 i is negation of y1
dataframe.loc[:, ('y2')] = dataframe['y1'] == 0

#convert true/false to 1/0
dataframe.loc[:, ('y2')] = dataframe['y2'].astype(int)

In [10]:
dataframe

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,1,0
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


In [15]:
#step 3 : prepare data for tensorflow (tensors)
#convert features to tensor
inputX = dataframe.loc[:, ['area', 'bathrooms']].as_matrix()
#convert labels to tensors
inputY = dataframe.loc[:, ['y1', 'y2']].as_matrix()

In [16]:
inputX

array([[  2.10400000e+03,   3.00000000e+00],
       [  1.60000000e+03,   3.00000000e+00],
       [  2.40000000e+03,   3.00000000e+00],
       [  1.41600000e+03,   2.00000000e+00],
       [  3.00000000e+03,   4.00000000e+00],
       [  1.98500000e+03,   4.00000000e+00],
       [  1.53400000e+03,   3.00000000e+00],
       [  1.42700000e+03,   3.00000000e+00],
       [  1.38000000e+03,   3.00000000e+00],
       [  1.49400000e+03,   3.00000000e+00]])

In [17]:
inputY

array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]])

In [18]:
#step 4 - write our hyper parameters
learning_rate = 0.000001
training_epochs = 2000
display_step = 50
n_samples = inputY.size

In [27]:
#step 5 : create our computational graph / neural network
#for feature input tensor, none means any no of examples
#2 because we have 2 features
#placeholder are our gateway for data into our computation graph
x = tf.placeholder(tf.float32, [None,2])

#create weights
#wx2 float matrix thet will keep updating 
#troughout training
#variables in tf hold and update parameters
#in memory buffer containing tensors
w = tf.Variable(tf.zeros([2,2]))

#add biases
b = tf.Variable(tf.zeros([2]))

#multiply our weights by our inputs
#weights are how we govern data flow on our computation graph
#multiply input by weights and add biasis
y_values = tf.add(tf.matmul(x, w), b)

#apply softmax to values we just created
#softmax is an activation function
#it normalizes our values
y = tf.nn.softmax(y_values)

#feed in a matrix of labels
y_ = tf.placeholder(tf.float32, [None, 2])

In [28]:
#Step 6 : perform training
#create our cost function, mean squared error
#reduce sum compute the sum of elements across dimension of a tensor
cost = tf.reduce_sum(tf.pow(y_ - y, 2))/(2*n_samples)

#gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [32]:
#initialize our variables and tensorflow sessions
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [38]:
#training loop
for i in range(training_epochs):
    sess.run(optimizer, feed_dict={x: inputX, y_: inputY})
    
    #write logs of training
    if(i) % display_step == 0:
        cc = sess.run(cost, feed_dict = {x: inputX, y_ : inputY})
        print("Training step:", '%04d' % (i), "cost= ", "{:.9f}".format(cc))
        
print("Optimized finished!")
training_cost = sess.run(cost, feed_dict= {x: inputX, y_ : inputY})
print("Training cost:",training_cost, "W= ", sess.run(w), "b= ", sess.run(b))

Training step: 0000 cost=  0.087686285
Training step: 0050 cost=  0.087686256
Training step: 0100 cost=  0.087686211
Training step: 0150 cost=  0.087686166
Training step: 0200 cost=  0.087686121
Training step: 0250 cost=  0.087686084
Training step: 0300 cost=  0.087686040
Training step: 0350 cost=  0.087686002
Training step: 0400 cost=  0.087685958
Training step: 0450 cost=  0.087685913
Training step: 0500 cost=  0.087685868
Training step: 0550 cost=  0.087685838
Training step: 0600 cost=  0.087685786
Training step: 0650 cost=  0.087685741
Training step: 0700 cost=  0.087685704
Training step: 0750 cost=  0.087685660
Training step: 0800 cost=  0.087685622
Training step: 0850 cost=  0.087685585
Training step: 0900 cost=  0.087685540
Training step: 0950 cost=  0.087685503
Training step: 1000 cost=  0.087685451
Training step: 1050 cost=  0.087685414
Training step: 1100 cost=  0.087685369
Training step: 1150 cost=  0.087685339
Training step: 1200 cost=  0.087685287
Training step: 1250 cost=

In [39]:
sess.run(y, feed_dict = {x: inputX})

array([[ 0.86096656,  0.13903348],
       [ 0.8000738 ,  0.1999262 ],
       [ 0.88891912,  0.11108083],
       [ 0.77331495,  0.22668508],
       [ 0.93084455,  0.06915545],
       [ 0.84819329,  0.15180665],
       [ 0.79077178,  0.2092282 ],
       [ 0.77502322,  0.22497673],
       [ 0.76784503,  0.23215498],
       [ 0.78498119,  0.21501882]], dtype=float32)